# 13. Исключения

Исключения возникают тогда, когда в программе возникает некоторая **исключительная ситуация**. Например, попытка чтения несуществующего файла.


## 13.1 Примеры

In [90]:
def exception_example(x):
    print(x)
    x = int(x)
    print(x)

exception_example(1)
exception_example('1')
exception_example('string')

1
1
1
1
string


ValueError: invalid literal for int() with base 10: 'string'

In [92]:
x = 1
x.isdigit()

AttributeError: 'int' object has no attribute 'isdigit'

In [6]:
data = {'x': 1, 'y': 2}
data['x']
data['z']

KeyError: 'z'

## 13.2 Обработка исключений

In [3]:
data = {'x': 1, 'y': 2}

try:
    value = data['z']
    print(1)
except (KeyError, TypeError):
    print('Oops, there is no z key in this dictionary, setting default')
    value = 2
else:
    print('everything works properly, z is in place')


# bad
x = 1
x += 1
try:
    value = data['z']
    # write to file
    print(1)
except Exception:
    print('Oops, there is no z key in this dictionary, setting default')
    value = 2
else:
    print('everything works properly, z is in place')


if 'z' not in data:
    value = 2

value = data.get('z', 2)
print('value', value)

Oops, there is no z key in this dictionary, setting default
Oops, there is no z key in this dictionary, setting default
value 2


## 12.3 Пользовательские исключения
https://docs.python.org/3/library/exceptions.html#exception-hierarchy

In [1]:
class ShortInputException(ValueError): 
    '''Пользовательский класс исключения.''' 
    def __init__(self, length, atleast):
        super().__init__(self)
        self.length = length
        self.atleast = atleast

try:
    text = input('Enter anything --> ') 
    x = int(text)
    
    if len(text) < 3:
        raise ShortInputException(len(text), 3)
except ShortInputException as ex:
    print(f'ShortInputException: ops value is too short, {ex.length} of {ex.atleast}') 
except ValueError:
    print('Text is not a number')
else:
    print('Everything is fine.')

Enter anything -->  2


ShortInputException: ops value is too short, 1 of 3


In [11]:
try:
    f = open('zen.txt')
    while True: 
        line = f.readline() 

        if len(line) == 0:
            break
        print(line, end='')
finally: 
    f.close()
    print('File is closed now')

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!File is closed now


## 13.4 Контекстный менеджер (with)

Типичной схемой является запрос некоторого ресурса в блоке try с последующим освобождением этого ресурса в блоке finally. Для того, чтобы сделать это более «чисто», существует оператор with.

Документация: https://docs.python.org/3/library/contextlib.html


In [3]:
with open('zen.txt') as f: 
    for line in f:
        print(line, end='')
print('File is already closed')

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!File is already closed


#### Пример использования контекста

In [8]:
from decimal import Decimal, localcontext

with localcontext() as context:
    context.prec = 56
    print(Decimal("1") / Decimal("28"))

0.035714285714285714285714285714285714285714285714285714286


In [15]:
print(Decimal("1") / Decimal("28"))

0.03571428571428571428571428571


#### Пример использования в тестах

In [31]:
import re

class InvalidPhoneFormatException(ValueError):
    pass

    
def parse_phone(phone: str , is_plus_required=True) -> str:
    """Возвращает номер телефона в формате 7XXXXXXXXXX или +7XXXXXXXXXX"""
    if is_plus_required and not phone.startswith('+'):
        raise InvalidPhoneFormatException(f'Invalid phone format: {phone}')
    

    pattern = r'[^\+\d]' if is_plus_required else r'[^\d]'
    return re.sub(pattern, '', phone)


In [14]:
import ipytest
ipytest.autoconfig()

In [36]:
%%ipytest -qq
import pytest

def test_parse_phone():
    # Act & Assert
    assert parse_phone('+7(910)510-07-00') == '+79105100700'

def test_parse_phone__no_plus__raise_exception():
    # Act & Assert
    with pytest.raises(InvalidPhoneFormatException) as ex:
        parse_phone('7(910)510-07-00')

    assert ex.value.args[0] == 'Invalid phone format: 7(910)510-07-00'

..                                                                                           [100%]


## 13.5 Реализация контекстного менеджера

In [41]:
import time

class Timer:
    def __enter__(self):
        self.start = time.perf_counter()
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.stop = time.perf_counter()
        self.total = self.stop - self.start

In [45]:
t = Timer()
with t:
    result = sum(range(100_500_000))

print(f'Total time spent: {t.total:.2f} sec')

Total time spent: 1.67 sec


#### Контекстное определение переменных окружения

In [46]:
import os

class set_env_var:
    def __init__(self, var_name, new_value):
        self.var_name = var_name
        self.new_value = new_value

    def __enter__(self):
        self.original_value = os.environ.get(self.var_name)
        os.environ[self.var_name] = self.new_value

    def __exit__(self, exc_type, exc_val, exc_tb):
        if self.original_value is None:
            del os.environ[self.var_name]
        else:
            os.environ[self.var_name] = self.original_value

In [48]:
os.environ['active_user'] = 'Petr'

In [55]:
print(f'Active user: {os.environ["active_user"]}')
with set_env_var('active_user', 'Ivan'):
    print(f'Active context user: {os.environ["active_user"]}')

print(f'Active context user after context: {os.environ["active_user"]}')

Active user: Petr
Active context user: Ivan
Active context user after context: Petr


#### Пример реализации конекстного менеджера через функцию-генератор

In [76]:
from contextlib import contextmanager
import os


@contextmanager
def set_env_var(var_name, new_value):
    original_value = os.environ.get(var_name)
    os.environ[var_name] = new_value
    try:
        yield
    finally:
        if original_value is None:
            del os.environ[var_name]
        else:
            os.environ[var_name] = original_value

In [78]:
print(f'Active user: {os.environ["active_user"]}')
with set_env_var('active_user', 'Ivan'):
    print(f'Active context user: {os.environ["active_user"]}')

print(f'Active context user after context: {os.environ["active_user"]}')

Active user: Petr
Active context user: Ivan
Active context user after context: Petr


In [66]:
from contextlib import contextmanager

@contextmanager
def hello_context_manager():
     print("Entering the context...")
     yield 12
     print("Leaving the context...")


with hello_context_manager() as hello:
     print(hello)


Entering the context...
12
Leaving the context...


### Пример работы с файлом

In [79]:
from contextlib import contextmanager

@contextmanager
def writable_file(file_path):
    file = open(file_path, mode="w")
    try:
        yield file
    finally:
        file.close()


In [88]:
users = ['Ivan', 'Petr']
with writable_file('users.txt') as file:
    file.write('\n'.join(users))
